In [0]:
import gurobipy as gp
from gurobipy import GRB
from InputReader import importlist 
import numpy as np
import time


def Supernal(inputfile):
    
    start = time.time()
    
    n, b, c, a = importlist(inputfile)
    
    NDF = []
    
    temp_a = []
    for i in range(n):
        temp = []
        for j in range(len(a)):
            temp.append(a[j][i])
        temp_a.append(temp)
            
    c_orig = c.copy()
    J = len(c)
    temp_c = [item for sublist in c for item in sublist]
    c = np.array(temp_c)
    a = np.array(temp_a)
    b = np.array(b)
    
    regions1 = [[0,0,0]]
    regions2 = []
    allregions = []
    
    m = gp.Model("Supernal Method")
    
    Items=list(np.arange(0,n))
    Dimensions= list(np.arange(0,1))
    
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")
    
    
    
    
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j")
        
    m.addConstr(gp.quicksum(x[i]*c[i] for i in Items) <= 0, name = "z1")
    m.addConstr(gp.quicksum(x[i]*c[i+n] for i in Items) <= 0, name = "z2")
    
    
    if J == 3:
        l = [0.7,0.2,0.1]
        m.setObjective((gp.quicksum(x[i]*c[i] for i in Items)*l[0] + 
                        gp.quicksum(x[i]*c[i+n] for i in Items)*l[1] + 
                        gp.quicksum(x[i]*c[i+n*2] for i in Items)*l[2]), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[i+2*n] for i in Items) <= 0, name = "z3")
    
    elif J ==2:
        l = [0.7,0.3]
        m.setObjective((gp.quicksum(x[i]*c[i] for i in Items)*l[0] + gp.quicksum(x[i]*c[i+n] for i in Items)*l[1]), GRB.MINIMIZE)
    
    
    m.optimize()
    
    
    def deleteDominated(regions):
        new_regions = []
        for i in range(len(regions)):
            i_dominated = [True]*(len(regions))
            for j in range(len(regions)):
              
                for dim in range(len(regions[i])):
                    if regions[i][dim] > regions[j][dim]:
                        i_dominated[j] = False
                        break
            del i_dominated[i]
            if not any(i_dominated):
                new_regions.append(regions[i])
    
        return(new_regions)
        
    
    def optimize(region):
        
        m.getConstrByName('z1').RHS = region[0]
        m.getConstrByName('z2').RHS = region[1]
        
        if J == 3:
            m.getConstrByName('z3').RHS = region[2]
        
        m.optimize()
        
        Mat = np.zeros((1,n))
        i = 0
        j = 0
        
        try:
            for p in x:
                if j == Mat.shape[1]:
                    j = 0
                    i += 1
                Mat[i,j] = abs(x[p].x)
                j+=1
            
            z = []
            for i in range(len(c_orig)):
                z_temp = 0
                for j in range(n):
                    z_temp += c_orig[i][j]*Mat[0][j]
                z.append(z_temp)
    
            if z not in NDF:
                NDF.append(z)
            
            
            if J == 2:
    
                regions2.append([z[0]-1,region[1]])
                regions2.append([region[0],z[1]-1])         
                
            if J == 3:
                
                regions2.append([z[0]-1,region[1],region[2]])
                regions2.append([region[0],z[1]-1,region[2]])
                regions2.append([region[0],region[1],z[2]-1])                        
                
            return None
            
        except: 
            return None
        
    iterations = True
    while iterations:
        r = 0
        while r < len(regions1):
            optimize(regions1[r])
            r += 1
        allregions.append(regions1)
        regions1 = deleteDominated(regions2)
        regions2 = []
        if len(regions1) == 0:
            iterations = False
    
    t = time.time() - start
    return NDF, len(NDF), t